In [1]:
#!pip install tensorflow
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, LSTM,BatchNormalization,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2

# Lstm model after using encoder

In [2]:
input_step=25 #to be changed after to 10,10
output_step=50
fsize=64

In [110]:
X1=np.load("X11_"+str(input_step)+".npy")
X2=np.load("X22_"+str(input_step)+".npy")


In [111]:
Y1=np.load("Y1_"+str(output_step)+".npy",allow_pickle=True)
Y2=np.load("Y2_"+str(output_step)+".npy",allow_pickle=True)

In [112]:
X1=np.float32(X1)
X2=np.float32(X2)
Y1=np.float32(Y1)
Y2=np.float32(Y2)

In [113]:
X1=np.reshape(X1,(X1.shape[0],input_step,fsize))
X2=np.reshape(X2,(X2.shape[0],input_step,fsize))

In [114]:
X1.shape

(52316, 25, 64)

In [115]:
XT=np.concatenate((X1, X2), axis=0)

In [116]:
YT=np.concatenate((Y1, Y2), axis=0)

In [117]:
def createModel(inputShape,output):
# define model
    model = Sequential()
    model.add(LSTM(100, activation='relu',input_shape=inputShape,return_sequences=True))
    model.add(LSTM(100,activation='relu', return_sequences=False))
#     model.add(LSTM(64,activation='relu', return_sequences=False))
#     model.add(Dense(16, activation='linear'))
#     model.add(Dropout(0.3))
    model.add(Dense(output,activation="sigmoid"))

    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,loss='mae',metrics=['mae','mse',tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.MeanAbsolutePercentageError()])
    # model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mse'])
    return model

In [118]:
time_steps = input_step  # Number of time steps
feature_length = 64  # Length of each feature vector
input_shape = (time_steps, feature_length)

model=createModel(input_shape,output_step)

In [119]:
model.summary()

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_55 (LSTM)              (None, 25, 100)           66000     
                                                                 
 lstm_56 (LSTM)              (None, 100)               80400     
                                                                 
 dense_28 (Dense)            (None, 50)                5050      
                                                                 
Total params: 151,450
Trainable params: 151,450
Non-trainable params: 0
_________________________________________________________________


In [3]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [121]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error,mean_squared_error
# load your data
X = XT  # your features
y = YT  # your target variable

# set up the k-fold cross-validation object
k =5   # number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# initialize a list to store the validation accuracies
val_mae = []
val_mse = []
val_mape=[]

# loop over the k folds
for train_index, val_index in kf.split(X):
    # split the data into training and validation sets
    X_train, y_train = X[train_index], y[train_index]
    X_val, y_val = X[val_index], y[val_index]

    # train your model on the training set
    model.fit(X_train, y_train)

    # evaluate the model on the validation set and store the mse
    y_pred = model.predict(X_val)
    val_acc = mean_absolute_error(y_val, y_pred)
    val_mae.append(val_acc)
    val_acc = mean_squared_error(y_val, y_pred)
    val_mse.append(val_acc)
    val_acc = MAPE(y_val, y_pred)
    val_mape.append(val_acc)
    

# print the mean and standard deviation of the validation mse
print(f'Validation mae: {np.mean(val_mae):.5f} ± {np.std(val_mae):.5f}')
print(f'Validation mse: {np.mean(val_mse):.5f} ± {np.std(val_mse):.5f}')
print(f'Validation mape: {np.mean(val_mape):.5f} ± {np.std(val_mape):.5f}')

652/652 [==============================] - 12s 18ms/step
Validation mae: 0.00609 ± 0.00123
Validation mse: 0.00010 ± 0.00004
Validation mape: 0.64772 ± 0.13070


In [122]:
loss, mae,mse,rmse,mape = model.evaluate(X2, Y2)
print('Test loss:', loss)
print('Test MAE:', mae)
print('Test MSE:', mse)
print('Test RMSE:', rmse)
print('Test MAPE:', mape)

1623/1623 [==============================] - 28s 17ms/step - loss: 0.0044 - mae: 0.0044 - mse: 5.9308e-05 - root_mean_squared_error: 0.0077 - mean_absolute_percentage_error: 0.4721
Test loss: 0.00443765614181757
Test MAE: 0.00443765614181757
Test MSE: 5.930836050538346e-05
Test RMSE: 0.007701194379478693
Test MAPE: 0.472080260515213


In [86]:
model.save("lstm_model_withencoder_"+str(input_step)+".h5")

# CNN-LSTM model

In [13]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Flatten, Reshape, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
import tensorflow as tf


# Define CNN-LSTM model architecture
model2 = Sequential()

# Convolutional layers
model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(input_step, 6, 500)))
model2.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and reshape for LSTM
model2.add(Flatten())
model2.add(Reshape((22, 32)))

# LSTM layers
model2.add(LSTM(32, return_sequences=True))
model2.add(LSTM(32, return_sequences=False))


# Dense layer
model2.add(Dense(output_step, activation='sigmoid'))

# Compile model
opt = Adam(learning_rate=0.001)
model2.compile(loss='mean_absolute_error', optimizer=opt, metrics=['mae', 'mse', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsolutePercentageError()])




# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

# Model checkpoint
checkpoint = ModelCheckpoint("Cnn_lstm_model_"+str(input_step)+".h5", save_best_only=True, monitor='val_loss', mode='min')

# Print model summary
model2.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 23, 4, 32)         144032    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 2, 32)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 704)               0         
                                                                 
 reshape_1 (Reshape)         (None, 22, 32)            0         
                                                                 
 lstm_2 (LSTM)               (None, 22, 32)            8320      
                                                                 
 lstm_3 (LSTM)               (None, 32)                8320      
                                                      

In [5]:
X11=np.load("X1_"+str(input_step)+".npy")
X22=np.load("X2_"+str(input_step)+".npy")

In [6]:
Y1=np.load("Y1_"+str(output_step)+".npy",allow_pickle=True)
Y2=np.load("Y2_"+str(output_step)+".npy",allow_pickle=True)

In [7]:
X11.shape,X22.shape

((52316, 25, 6, 500), (51922, 25, 6, 500))

In [8]:
X11=np.float32(X11)
X22=np.float32(X22)

In [9]:
Y1=np.float32(Y1)
Y2=np.float32(Y2)

In [10]:
XT=np.concatenate((X11, X22), axis=0)

In [11]:
YT=np.concatenate((Y1, Y2), axis=0)

In [24]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error,mean_squared_error
import gc
X = XT  # your features
y = YT  # your target variable

# set up the k-fold cross-validation object
k =5   # number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# initialize a list to store the validation accuracies
val_mae = []
val_mse = []
val_mape=[]

# loop over the k folds
for train_index, val_index in kf.split(X):
    # split the data into training and validation sets
    X_train, y_train = X[train_index], y[train_index]
    X_val, y_val = X[val_index], y[val_index]

    # train your model on the training set
    model2.fit(X_train, y_train)

    # evaluate the model on the validation set and store the mse
    y_pred = model2.predict(X_val)
    val_acc = mean_absolute_error(y_val, y_pred)
    val_mae.append(val_acc)
    val_acc = mean_squared_error(y_val, y_pred)
    val_mse.append(val_acc)
    val_acc = MAPE(y_val, y_pred)
    val_mape.append(val_acc)
    del X_train,y_train,X_val,y_val
    gc.collect()
    

# print the mean and standard deviation of the validation mse
print(f'Validation mae: {np.mean(val_mae):.5f} ± {np.std(val_mae):.5f}')
print(f'Validation mse: {np.mean(val_mse):.5f} ± {np.std(val_mse):.5f}')
print(f'Validation mape: {np.mean(val_mape):.5f} ± {np.std(val_mape):.5f}')

652/652 [==============================] - 10s 15ms/step
Validation mae: 0.00798 ± 0.00253
Validation mse: 0.00017 ± 0.00012
Validation mape: 0.84733 ± 0.26945


In [23]:
loss, mae,mse,rmse,mape = model2.evaluate(X22, Y2)
print('Test loss:', loss)
print('Test MAE:', mae)
print('Test MSE:', mse)
print('Test RMSE:', rmse)
print('Test MAPE:', mape)

1719/1719 [==============================] - 8s 4ms/step - loss: 0.0039 - mae: 0.0039 - mse: 3.7264e-05 - root_mean_squared_error: 0.0061 - mean_absolute_percentage_error: 0.4098
Test loss: 0.0038862728979438543
Test MAE: 0.0038862728979438543
Test MSE: 3.7264293496264145e-05
Test RMSE: 0.0061044516041874886
Test MAPE: 0.4098218083381653


In [25]:
model2.save("Cnn_lstm_model_"+str(input_step)+".h5")